In [ ]:
from PIL import Image
import numpy as np

img  =  Image.open("pics/1.jpg")
img_np =  np.asarray(img)

In [ ]:
img_np.shape

(3024, 4032, 3)

In [ ]:
w, h = img.size
compress_rate = 0.05
img_resize = img.resize((500, 500))

In [ ]:
img_resize_np =  np.asarray(img_resize)
img_resize_np.shape

(500, 500, 3)

In [ ]:
img_np[0][0]

array([169, 193, 231], dtype=uint8)

In [ ]:
img_resize_np[0][0]

array([167, 191, 230], dtype=uint8)

In [ ]:
import pandas as pd
dmc_rgb = pd.read_csv("DMC_RGB.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 81, saw 2


In [ ]:
pd.read_csv("DMC_RGB.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 81, saw 2


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=768076c7-65ce-42d5-b86d-074f9f75d785' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>